This tutorial is generated from a [Jupyter](http://jupyter.org/) notebook that can be found [here](https://github.com/elfi-dev/notebooks). 

## BOLFI for the daycare example

Aim: to see how the tuning parameters acq_noise_var and update_interval affect performance of BOLFI

In [1]:
import numpy as np
import scipy.stats
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
%precision 2

import logging
logging.basicConfig(level=logging.INFO)

# Set an arbitrary global seed to keep the randomly generated quantities the same
seed = 1
np.random.seed(seed)

import elfi

In [2]:
from elfi.examples import daycare
model = daycare.get_model(seed_obs=seed)
#true_params = [3.6, 0.6, 0.1]
# priors.append(elfi.Prior('uniform', 0, 11, model=m, name='t1'))
# priors.append(elfi.Prior('uniform', 0, 2, model=m, name='t2'))
# priors.append(elfi.Prior('uniform', 0, 1, model=m, name='t3'))

INFO:root:Generated observations with true parameters t1: 3.6, t2: 0.600, t3: 0.1, 


### Fitting the surrogate model

In [3]:
log_d = elfi.Operation(np.log, model['d'])

As BOLFI is a more advanced inference method, its interface is also a bit more involved as compared to for example rejection sampling. But not much: Using the same graphical model as earlier, the inference could begin by defining a Gaussian process (GP) model, for which ELFI uses the [GPy](https://sheffieldml.github.io/GPy/) library. This could be given as an `elfi.GPyRegression` object via the keyword argument `target_model`. In this case, we are happy with the default that ELFI creates for us when we just give it each parameter some `bounds` as a dictionary.

Other notable arguments include the `initial_evidence`, which gives the number of initialization points sampled straight from the priors before starting to optimize the acquisition of points, `update_interval` which defines how often the GP hyperparameters are optimized, and `acq_noise_var` which defines the diagonal covariance of noise added to the acquired points. Note that in general BOLFI does not benefit from a `batch_size` higher than one, since the acquisition surface is updated after each batch (especially so if the noise is 0!).

In [4]:
bolfi = elfi.BOLFI(log_d, batch_size=1, initial_evidence=20, update_interval=10, 
                   bounds={'t1':(0, 11), 't2':(0, 2), 't3':(0, 1)}, acq_noise_var=0.1, seed=seed)

Sometimes you may have some samples readily available. You could then initialize the GP model with a dictionary of previous results by giving `initial_evidence=result.outputs`.

The BOLFI class can now try to `fit` the surrogate model (the GP) to the relationship between parameter values and the resulting discrepancies. We'll request only 100 evidence points (including the `initial_evidence` defined above).

In [5]:
%time post = bolfi.fit(n_evidence=200)

INFO:elfi.methods.inference.bolfi:BOLFI: Fitting the surrogate model...


Progress [==================================================] 100.0% Complete


INFO:elfi.methods.posteriors:Using optimized minimum value (-3.1782) of the GP discrepancy mean function as a threshold


CPU times: user 10min 12s, sys: 195 ms, total: 10min 13s
Wall time: 10min 18s


In [6]:
bolfi.target_model


Name : GP regression
Objective : 226.49250335260248
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  GP_regression.           |                   value  |  constraints  |      priors    
  sum.rbf.variance         |      0.5822645575495805  |      +ve      |  Ga(0.00014, 1)
  sum.rbf.lengthscale      |      0.2036598643441625  |      +ve      |    Ga(3.7, 1)  
  sum.bias.variance        |     0.35323146351618473  |      +ve      |  Ga(3.6e-05, 1)
  Gaussian_noise.variance  |  1.3268833713003107e-05  |      +ve      |                

In [7]:
# bolfi.target_model._gp.plot() can't plot for 3D

TypeError: calculated free_dims [0 1 2] from visible_dims None and fixed_dims [] is neither 1D nor 2D

It may be useful to see the acquired parameter values and the resulting discrepancies:

In [ ]:
bolfi.plot_state();

In [ ]:
bolfi.plot_discrepancy();

There could be an unnecessarily high number of points at parameter bounds. These could probably be decreased by lowering the covariance of the noise added to acquired points, defined by the optional `acq_noise_var` argument for the BOLFI constructor. Another possibility could be to [add virtual derivative observations at the borders](https://arxiv.org/abs/1704.00963), though not yet implemented in ELFI.

### BOLFI Posterior

In [ ]:
post2 = bolfi.extract_posterior(-1.)

In [ ]:
#post.plot(logpdf=True)

### Sampling

Finally, samples from the posterior can be acquired with an MCMC sampler. By default it runs 4 chains, and half of the requested samples are spent in adaptation/warmup. Note that depending on the smoothness of the GP approximation, the number of priors, their gradients etc., **this may be slow**.

In [ ]:
%time result_BOLFI = bolfi.sample(1000, info_freq=1000)

Cannot sample from the posterior? to be investigated

In [ ]:
result_BOLFI

In [ ]:
result_BOLFI.plot_traces();

The black vertical lines indicate the end of warmup, which by default is half of the number of iterations.

In [ ]:
result_BOLFI.plot_marginals();